In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis

/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis


In [3]:
import pandas as pd
from model.LSTM import LSTM_model
from preprocessing.text_preprocessing import Sentences2LemmaWords
from preprocessing.text_preprocessing import GetUnique
from preprocessing.text_preprocessing import setTokenizer
from preprocessing.text_preprocessing import LemmaWords2Seqeunces
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time
from log.logger import Logger

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
train = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/train.tsv', sep='\t')
test = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/test.tsv', sep='\t')

print('train data shape: ', train.shape)
print('test data shape: ', test.shape)
print('----------[train infomation]----------')
print(train.info())
print('----------[test infomation]----------')
print(test.info())

train data shape:  (156060, 4)
test data shape:  (66292, 3)
----------[train infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB
None
----------[test infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    66292 non-null  int64 
 1   SentenceId  66292 non-null  int64 
 2   Phrase      66292 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB
None


In [5]:
train_tokens = Sentences2LemmaWords(train.Phrase)
test_tokens = Sentences2LemmaWords(test.Phrase)

2020-07-14 12:13:03,846[DEBUG|preprocessing, 17] Sentence2LemmaWords: start
/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/preprocessing/text_preprocessing.py:23: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 23 of the file /content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/preprocessing/text_preprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  sentence = BeautifulSoup(sentence).get_text()


235 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


10725 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'log'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


...

155869 / 156060 Phrase

2020-07-14 12:15:35,966[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 152.12680172920227sec
2020-07-14 12:15:35,968[DEBUG|preprocessing, 17] Sentence2LemmaWords: start


156060 / 156060 Phrase
66249 / 66292 Phrase

2020-07-14 12:16:39,463[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 63.49460530281067sec


66292 / 66292 Phrase


In [6]:
unique_words, maxlen = GetUnique(train_tokens)
print(len(unique_words), maxlen)

2020-07-14 12:16:39,478[DEBUG|preprocessing, 44] GetUnique: start
2020-07-14 12:16:39,608[DEBUG|preprocessing, 56] GetUnique: finish. 계산 시간 0.12943553924560547sec


13745 48


In [7]:
target = train.Sentiment
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(train_tokens, y_target,test_size=0.2, stratify=y_target)

In [9]:
tokenizer = setTokenizer(x_train, len(unique_words))
x_train = LemmaWords2Seqeunces(x_train, tokenizer, maxlen)
x_valid = LemmaWords2Seqeunces(x_valid, tokenizer, maxlen)
x_test = LemmaWords2Seqeunces(test_tokens, tokenizer, maxlen)

2020-07-14 12:16:40,756[DEBUG|preprocessing, 63] setTokenizer: start
2020-07-14 12:16:41,679[DEBUG|preprocessing, 70] setTokenizer: finish. 계산 시간 0.9232323169708252sec
2020-07-14 12:16:41,684[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 12:16:42,885[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 1.2010018825531006sec
2020-07-14 12:16:42,891[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 12:16:43,486[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.5954606533050537sec
2020-07-14 12:16:43,491[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 12:16:44,030[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.5382378101348877sec


In [10]:
x_test.shape

(66292, 48)

In [28]:
from model.LSTM import LSTM_model
model = LSTM_model(len(unique_words), maxlen, num_classes)

logger = Logger('main')
start_time = time.time()
logger.debug('fit start')

model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=5,batch_size=2048, verbose=1)

fit_time = time.time() - start_time
logger.debug('fit end.fit_time{}'.format(fit_time))

2020-07-14 12:32:17,825[DEBUG|LSTM_model_generate, 9] LSTM_model: start
2020-07-14 12:32:18,095[DEBUG|LSTM_model_generate, 20] LSTM_model: finish
2020-07-14 12:32:18,097[DEBUG|main, 6] fit start
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 16s 124us/step - loss: 1.2077 - accuracy: 0.5255 - val_loss: 1.0477 - val_accuracy: 0.5827
Epoch 2/5
124848/124848 [==============================] - 15s 120us/step - loss: 0.9332 - accuracy: 0.6230 - val_loss: 0.8778 - val_accuracy: 0.6457
Epoch 3/5
124848/124848 [==============================] - 15s 120us/step - loss: 0.7856 - accuracy: 0.6792 - val_loss: 0.8405 - val_accuracy: 0.6595
Epoch 4/5
124848/124848 [==============================] - 15s 119us/step - loss: 0.7269 - accuracy: 0.7012 - val_loss: 0.8318 - val_accuracy: 0.6617
Epoch 5/5
124848/124848 [==============================] - 15s 122us/step - loss: 0.6902 - accuracy: 0.7163 - val_loss: 0.8311 - val_accuracy: 0.6636


2020-07-14 12:33:34,694[DEBUG|main, 11] fit end.fit_time76.59631443023682


In [29]:
y_pred = model.predict_classes(x_test)

submission = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/sampleSubmission.csv')

In [30]:
x_test.shape

(66292, 48)

In [31]:
y_pred.shape

(66292,)

In [32]:
submission.shape

(66292, 2)

In [33]:
submission['Sentiment'] = y_pred

In [34]:
submission.to_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/submission_LSTM.csv')